In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
import numpy as np
import json
from tqdm.auto import tqdm
import random
import transformers

import os
import sys
sys.path.append('..')

from utils import model_utils
from baukit import nethook
from operator import itemgetter

from transformers import AutoModelForCausalLM, AutoTokenizer


In [4]:

MODEL_NAME =  "gpt2-medium" #"facebook/galactica-6.7b"  # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, low_cpu_mem_usage=True, #torch_dtype=torch_dtype,
    device_map = "balanced"
)

# nethook.set_requires_grad(False, model)
# model.eval().cuda()

print(f"{MODEL_NAME} ==> device: {model.device}, memory: {model.get_memory_footprint()}")

gpt2-medium ==> device: cuda:0, memory: 1444458592


In [5]:
tokenizer.pad_token = tokenizer.eos_token
nethook.set_requires_grad(True, model)

In [6]:
model.config.n_layer

24

In [7]:
for n, p in model.named_parameters():
    print(n, p.shape, p.device)

transformer.wte.weight torch.Size([50257, 1024]) cuda:0
transformer.wpe.weight torch.Size([1024, 1024]) cuda:0
transformer.h.0.ln_1.weight torch.Size([1024]) cuda:0
transformer.h.0.ln_1.bias torch.Size([1024]) cuda:0
transformer.h.0.attn.c_attn.weight torch.Size([1024, 3072]) cuda:0
transformer.h.0.attn.c_attn.bias torch.Size([3072]) cuda:0
transformer.h.0.attn.c_proj.weight torch.Size([1024, 1024]) cuda:0
transformer.h.0.attn.c_proj.bias torch.Size([1024]) cuda:0
transformer.h.0.ln_2.weight torch.Size([1024]) cuda:0
transformer.h.0.ln_2.bias torch.Size([1024]) cuda:0
transformer.h.0.mlp.c_fc.weight torch.Size([1024, 4096]) cuda:0
transformer.h.0.mlp.c_fc.bias torch.Size([4096]) cuda:0
transformer.h.0.mlp.c_proj.weight torch.Size([4096, 1024]) cuda:0
transformer.h.0.mlp.c_proj.bias torch.Size([1024]) cuda:0
transformer.h.1.ln_1.weight torch.Size([1024]) cuda:0
transformer.h.1.ln_1.bias torch.Size([1024]) cuda:0
transformer.h.1.attn.c_attn.weight torch.Size([1024, 3072]) cuda:0
transfor

In [8]:
input_text = "Facebook is a"

model_utils.generate_fast(
    model, tokenizer, 
    [input_text], 
    argmax_greedy=True,
    debug=True
)

' great'[1049] -- 10.643129348754883 [' great'(10.643) ' social'(4.839) ' wonderful'(2.623) ' platform'(2.568) ' place'(2.216) ]
' place'[1295] -- 26.1378173828125 [' place'(26.138) ' way'(14.685) ' tool'(9.537) ' platform'(8.624) ' example'(3.025) ]
' to'[284] -- 76.33644104003906 [' to'(76.336) ' for'(17.394) ','(1.438) '.'(1.046) ' where'(0.745) ]
' find'[1064] -- 13.001508712768555 [' find'(13.002) ' start'(7.52) ' share'(6.774) ' get'(5.728) ' learn'(3.726) ]
' out'[503] -- 9.319870948791504 [' out'(9.32) ' the'(3.948) ' information'(3.422) ' a'(3.387) ' great'(3.064) ]
' what'[644] -- 29.852310180664062 [' what'(29.852) ' about'(22.128) ' more'(20.307) ' how'(7.552) ' if'(2.388) ]
''s'[338] -- 28.730690002441406 [''s'(28.731) ' other'(9.443) ' people'(7.209) ' is'(4.327) ' the'(4.202) ]
' going'[1016] -- 36.31201934814453 [' going'(36.312) ' happening'(22.712) ' new'(8.658) ' trending'(4.209) ' on'(2.813) ]
' on'[319] -- 99.1840591430664 [' on'(99.184) ' down'(0.182) ' in'(0.157)

(["Facebook is a great place to find out what's going on in the world. It's a great"],
 {'past_key_values': ((tensor([[[[ 0.1553,  0.0996,  0.1718,  ..., -0.5769, -0.1995,  0.1173],
              [ 1.1678,  0.2322,  0.3601,  ...,  0.2506,  0.2609, -0.3677],
              [ 1.3599,  0.2809, -0.0502,  ...,  1.8119,  0.3128, -0.4774],
              ...,
              [-0.7588,  0.3667,  0.1868,  ...,  2.4225, -0.2204, -0.7005],
              [-0.3104,  0.4202,  0.2892,  ...,  1.3045,  0.0147, -0.6919],
              [ 1.0475,  0.1504, -0.1856,  ...,  2.8886,  0.1214, -0.4535]],
    
             [[-0.1601, -0.6617,  0.4319,  ..., -0.0878, -0.1617, -0.2335],
              [ 0.0163, -0.0235, -0.2808,  ..., -0.2326, -0.3399, -0.5331],
              [-0.0516,  0.6636, -1.5486,  ..., -0.8864, -0.7555, -1.3827],
              ...,
              [ 0.6449, -0.6812,  0.1334,  ...,  0.4841, -1.2640,  1.2577],
              [-0.3053, -0.4646, -0.1761,  ...,  0.4556, -0.0671, -0.6984],
              

In [9]:
tokenized = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(tokenized.input_ids.to(model.device), max_new_tokens=30, top_k = 1)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Facebook is a great place to find out what's going on in the world. It's a great place to find out what's going on in the world.



In [10]:
mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=True)

model2 = mt.model
tokenizer2 = mt.tokenizer
tokenizer2.pad_token = tokenizer2.eos_token

print(f"{MODEL_NAME} ==> device: {model2.device}, memory: {model2.get_memory_footprint()}")

gpt2-medium ==> device: cuda:0, memory: 1444458592


In [22]:
model_utils.generate_fast(
    model2, tokenizer2, 
    [input_text], 
    argmax_greedy=True,
    debug=True,
    max_new_tokens=30
)

' great'[1049] -- 10.643129348754883 [' great'(10.643) ' social'(4.839) ' wonderful'(2.623) ' platform'(2.568) ' place'(2.216) ]
' place'[1295] -- 26.1378173828125 [' place'(26.138) ' way'(14.685) ' tool'(9.537) ' platform'(8.624) ' example'(3.025) ]
' to'[284] -- 76.33644104003906 [' to'(76.336) ' for'(17.394) ','(1.438) '.'(1.046) ' where'(0.745) ]
' find'[1064] -- 13.001508712768555 [' find'(13.002) ' start'(7.52) ' share'(6.774) ' get'(5.728) ' learn'(3.726) ]
' out'[503] -- 9.319870948791504 [' out'(9.32) ' the'(3.948) ' information'(3.422) ' a'(3.387) ' great'(3.064) ]
' what'[644] -- 29.852310180664062 [' what'(29.852) ' about'(22.128) ' more'(20.307) ' how'(7.552) ' if'(2.388) ]
''s'[338] -- 28.730690002441406 [''s'(28.731) ' other'(9.443) ' people'(7.209) ' is'(4.327) ' the'(4.202) ]
' going'[1016] -- 36.31201934814453 [' going'(36.312) ' happening'(22.712) ' new'(8.658) ' trending'(4.209) ' on'(2.813) ]
' on'[319] -- 99.1840591430664 [' on'(99.184) ' down'(0.182) ' in'(0.157)

(["Facebook is a great place to find out what's going on in the world. It's a great place to find out what's going on in the world. "],
 {'past_key_values': ((tensor([[[[ 0.1553,  0.0996,  0.1718,  ..., -0.5769, -0.1995,  0.1173],
              [ 1.1678,  0.2322,  0.3601,  ...,  0.2506,  0.2609, -0.3677],
              [ 1.3599,  0.2809, -0.0502,  ...,  1.8119,  0.3128, -0.4774],
              ...,
              [ 1.1604,  0.2513, -0.1776,  ...,  2.7598,  0.2482, -0.5414],
              [-0.5904,  0.1113, -0.2870,  ...,  1.4157, -0.3826, -0.2942],
              [-1.2011,  0.2593, -0.1598,  ..., -0.9868, -0.5850, -0.6153]],
    
             [[-0.1601, -0.6617,  0.4319,  ..., -0.0878, -0.1617, -0.2335],
              [ 0.0163, -0.0235, -0.2808,  ..., -0.2326, -0.3399, -0.5331],
              [-0.0516,  0.6636, -1.5486,  ..., -0.8864, -0.7555, -1.3827],
              ...,
              [-0.2382,  0.8749, -2.0632,  ..., -1.0666, -0.7629, -1.9213],
              [-0.4626, -0.3911,  0.7434,

In [12]:
def get_zero_out_intervention(zero_out_layer):
    def zero_out_intervention(output, layer):
        if(layer == zero_out_layer):
            x = output[0]
            x = torch.zeros(output[0].shape).to(output[0].dtype).to(output[0].device)
            output = (x, output[1])
        return output
    return zero_out_intervention

In [13]:
with nethook.TraceDict(
    model,
    mt.layer_names,
    # edit_output= get_zero_out_intervention(zero_out_layer="transformer.h.11")
) as traces:
    outputs = model(**tokenized)

In [14]:
model_utils.check_structure_tree(traces["transformer.h.11"].output)

<class 'tuple'> [2] 
├───0: <class 'torch.Tensor'>  [torch.Size([1, 3, 1024])] cuda:0
└───1: <class 'tuple'> [2] 
    ├───0: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0
    └───1: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0


In [15]:
traces["transformer.h.11"].output

(tensor([[[-0.1469,  0.7708,  1.4290,  ...,  0.5000, -0.6622, -0.2038],
          [-0.2816,  4.3047,  3.7451,  ..., -1.9087, -2.5072,  2.6908],
          [-3.5915,  2.0401,  1.5464,  ..., -4.6442, -3.9636,  2.1443]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 (tensor([[[[-0.4604, -1.2155,  0.7317,  ..., -0.0582, -0.9350,  0.2144],
            [-1.3736, -1.7998,  2.0934,  ...,  0.4302,  1.1855, -1.9837],
            [-0.7040, -1.4722,  1.6294,  ..., -1.3447, -0.5740, -1.4226]],
  
           [[ 1.1540,  0.1938, -0.2520,  ..., -1.8779, -2.2169,  2.3769],
            [-1.1719,  0.2851, -0.6967,  ..., -1.2583,  0.5322,  2.1113],
            [-0.0514, -1.0756, -1.3377,  ...,  0.5335,  0.7444,  0.7725]],
  
           [[ 1.8807, -0.3222, -1.2747,  ..., -1.1843, -1.1302,  0.2447],
            [-0.1514, -1.8124,  3.3814,  ..., -0.1750,  4.2818,  0.8564],
            [ 0.6688, -0.6131,  4.9118,  ..., -1.1441,  2.5474,  1.1113]],
  
           ...,
  
           [[ 0.5369, -0.3169,  0.3

In [16]:
isinstance(traces["transformer.h.23"].output[0], torch.Tensor)

True

In [17]:
traces["transformer.h.23"].output[0].norm().backward()

In [18]:
model_utils.check_structure_tree(traces["transformer.h.11"].output)

<class 'tuple'> [2] 
├───0: <class 'torch.Tensor'>  [torch.Size([1, 3, 1024])] cuda:0
└───1: <class 'tuple'> [2] 
    ├───0: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0
    └───1: <class 'torch.Tensor'>  [torch.Size([1, 16, 3, 64])] cuda:0


In [19]:
traces["transformer.h.11"].output

(tensor([[[-0.1469,  0.7708,  1.4290,  ...,  0.5000, -0.6622, -0.2038],
          [-0.2816,  4.3047,  3.7451,  ..., -1.9087, -2.5072,  2.6908],
          [-3.5915,  2.0401,  1.5464,  ..., -4.6442, -3.9636,  2.1443]]],
        device='cuda:0', grad_fn=<AddBackward0>),
 (tensor([[[[-0.4604, -1.2155,  0.7317,  ..., -0.0582, -0.9350,  0.2144],
            [-1.3736, -1.7998,  2.0934,  ...,  0.4302,  1.1855, -1.9837],
            [-0.7040, -1.4722,  1.6294,  ..., -1.3447, -0.5740, -1.4226]],
  
           [[ 1.1540,  0.1938, -0.2520,  ..., -1.8779, -2.2169,  2.3769],
            [-1.1719,  0.2851, -0.6967,  ..., -1.2583,  0.5322,  2.1113],
            [-0.0514, -1.0756, -1.3377,  ...,  0.5335,  0.7444,  0.7725]],
  
           [[ 1.8807, -0.3222, -1.2747,  ..., -1.1843, -1.1302,  0.2447],
            [-0.1514, -1.8124,  3.3814,  ..., -0.1750,  4.2818,  0.8564],
            [ 0.6688, -0.6131,  4.9118,  ..., -1.1441,  2.5474,  1.1113]],
  
           ...,
  
           [[ 0.5369, -0.3169,  0.3

In [20]:
layer = nethook.get_module(model, "transformer.h.11")

In [21]:
layer.mlp.c_proj.weight.shape, layer.mlp.c_proj.weight.grad.shape

(torch.Size([4096, 1024]), torch.Size([4096, 1024]))